In [1]:
import pandas as pd
from datetime import datetime
import json

import warnings
warnings.filterwarnings("ignore")

fase = '06_evaluar_prediccion'

In [2]:
with open('gen_config.json', 'r') as file:
    gen_config =json.load(file)

In [3]:
folder = gen_config['folder']

path_prod_data = gen_config['path_prod_data']
path_pred_test = gen_config['path_pred_test']
path_pred_futuro = gen_config['path_pred_futuro']

path_kaggle = gen_config['path_kaggle']

print(f"{'COMIENZA':-^100}")
print(f"{fase:-^100}")

----------------------------------------------COMIENZA----------------------------------------------
---------------------------------------06_evaluar_prediccion----------------------------------------


In [4]:
def perdida(real, prediccion):
    total = real.sum()
    perdida =  abs(real - prediccion).sum()
    resultado = perdida / total
    return resultado

In [5]:
prod_data = pd.read_parquet(f"{folder}/{path_prod_data}")
prod_a_predecir = pd.read_csv('../productos_a_predecir.txt', sep='\t')
df_test = pd.read_parquet(f"{folder}/{path_pred_test}")
df_fut = pd.read_parquet(f"{folder}/{path_pred_futuro}")

In [6]:
df_pred_test = df_test[['product_id', 'tn_futuro_real', 'tn_prediccion_real']].groupby('product_id', as_index=False).sum()
df_pred_test['error'] = -(df_pred_test['tn_futuro_real'] - df_pred_test['tn_prediccion_real'])/df_pred_test['tn_futuro_real']*100
df_pred_test['error_abs'] = -(df_pred_test['tn_futuro_real'] - df_pred_test['tn_prediccion_real'])
df_pred_test['weight'] = df_pred_test['tn_futuro_real']/df_pred_test['tn_futuro_real'].sum()
df_pred_test.sort_values('weight', ascending=False, inplace=True)

perdida = perdida(df_pred_test['tn_futuro_real'], df_pred_test['tn_prediccion_real'])
print(f"Perdida en test: {perdida}")
print(f"Primeras filas de test:\n{df_pred_test.iloc[:11,:4]}")

Perdida en test: 0.014738442600539517
Primeras filas de test:
    product_id  tn_futuro_real  tn_prediccion_real     error
0        20001      1286.02350         1275.017985 -0.855779
1        20002      1002.49082         1001.522797 -0.096562
2        20003       680.22322          678.744574 -0.217377
3        20004       506.69680          514.766985  1.592705
4        20005       491.74883          491.918844  0.034573
31       20032       402.43544          394.125431 -2.064930
8        20009       360.53386          352.611753 -2.197327
5        20006       315.78911          307.242322 -2.706486
18       20019       287.41511          295.367618  2.766907
10       20011       241.39206          242.798390  0.582592
6        20007       231.85925          230.941209 -0.395948


In [7]:
# df_pred_ungroup = df_pred_test[['group_index', 'tn_prediccion_real']]
# df_pred_ungroup = df_pred_ungroup.merge(prod_data_ungroup, on='group_index', how='left')
# df_pred_ungroup = df_pred_ungroup[['group_index', 'product_id', 'tn_prediccion_real', 'proporcion']]

# df_pred_ungroup = df_pred_ungroup[df_pred_ungroup['product_id'].isin(prod_a_predecir['product_id'])]
# df_pred_ungroup['prediccion_final'] = df_pred_ungroup['tn_prediccion_real'] * df_pred_ungroup['proporcion']
# df_pred_ungroup.sort_values('product_id', ascending=True, inplace=True)

# df_pred_ungroup[df_pred_ungroup['group_index'] == 281]

In [8]:
df_pred_fut = df_fut[['product_id', 'tn_futuro_real', 'tn_prediccion_real']].groupby('product_id', as_index=False).sum()
#df_pred_fut['error'] = -(df_pred_fut['tn_futuro_real'] - df_pred_fut['tn_prediccion_real'])/df_pred_fut['tn_futuro_real']*100
#df_pred_fut['error_abs'] = -(df_pred_fut['tn_futuro_real'] - df_pred_fut['tn_prediccion_real'])
#df_pred_fut['weight'] = df_pred_fut['tn_futuro_real']/df_pred_fut['tn_futuro_real'].sum()
#df_pred_fut.sort_values('weight', ascending=False, inplace=True)

print(f"Primeras filas de futuro:\n{df_pred_fut.iloc[:11,:4]}")

Primeras filas de futuro:
    product_id  tn_futuro_real  tn_prediccion_real
0        20001             0.0         1364.216212
1        20002             0.0          863.417047
2        20003             0.0          751.680489
3        20004             0.0          522.599408
4        20005             0.0          552.310657
5        20006             0.0          357.536921
6        20007             0.0          462.251481
7        20008             0.0          249.814915
8        20009             0.0          373.989305
9        20010             0.0          418.310291
10       20011             0.0          391.895082


In [9]:
# df_fut_ungroup = df_fut[['group_index', 'tn_prediccion_real']]
# df_fut_ungroup = df_fut_ungroup.merge(prod_data_ungroup, on='group_index', how='left')
# df_fut_ungroup = df_fut_ungroup[['group_index', 'product_id', 'tn_prediccion_real', 'proporcion']]

# df_fut_ungroup = df_fut_ungroup[df_fut_ungroup['product_id'].isin(prod_a_predecir['product_id'])]
# df_fut_ungroup['prediccion_final'] = df_fut_ungroup['tn_prediccion_real'] * df_fut_ungroup['proporcion']
# df_fut_ungroup.sort_values('product_id', ascending=True, inplace=True)

In [10]:
kaggle =  df_pred_fut[['product_id','tn_prediccion_real']]
kaggle['tn_prediccion_real'] = kaggle['tn_prediccion_real'].clip(lower=0)
kaggle.columns = ['product_id', 'tn']
#kaggle.to_csv('kaggle-new.csv', index=False)
kaggle.to_csv(f"{folder}/{datetime.now().strftime('%Y%m%d_%H%M%S')}_{path_kaggle}", index=False)


In [11]:
#estado_control = f"06_evaluar_prediccion Terminado - {nombrefile} - {datetime.now()}"

In [12]:
print(f"{fase:-^100}")
print(f"{'FINALIZA':-^100}\n\n\n")

---------------------------------------06_evaluar_prediccion----------------------------------------
----------------------------------------------FINALIZA----------------------------------------------



